In [2]:
'''
Here I import the libraries I will need
'''

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep
import re
from selenium.common.exceptions import NoSuchElementException


In [79]:
'''
In the next cells, I provide the methods to find and record the required information from the html. I also provide
the try and except terms in order to prevent scraping failure.
'''

def extract_location_from_result(loc):
    try:
        for n in loc.findAll('a', {'data-automation':'jobLocation'}):
            location.append(n.text.strip())
            search.append(job)
    except:
        location.append(0)

In [80]:
def extract_jobtitle_from_result(jobtitle):
    try:
        for n in jobtitle.findAll('a', {'data-automation':'jobTitle'}):
            title.append(n.text.strip())
    except:
        title.append(0) 

In [81]:
def extract_company_from_result(comp):
    try:
        for n in comp.findAll('a', {'data-automation':'jobCompany'}):
            company.append(n.text.strip())
    except:
        company.append(0) 

In [152]:
def extract_salary_from_result(sal):
    try:
        for n in sal.findAll('span', {'data-automation':'jobSalary'}):
            salary.append(n.text)
        else:
            salary.append('Unknown')
    except:
        salary.append('Uknown')

In [153]:
def extract_links_from_result(url):
    try:
        for n in url.find_all('a', href=re.compile("searchrequesttoken")):
            links.append('www.seek.com.au' + n.get('href'))
    except:
        links.append('0') 

In [166]:
'''
This is the actual web-scraper part. I used selenium, but I would recommend using requests as selenium is somewhat
slow. I searched for 4 different job types, located in Sydney. The scraper searches through pages using the next
button, until the next button is no longer present.
'''

joi = ['Data Science', 'Data Analyst', 'Data Engineer', 'Business Intelligence']
links = []
title = []
post_date = []
salary = []
desc = []
company = []
search = []
location = []

url = "https://www.seek.com.au/"

driver = webdriver.Chrome(executable_path="./chromedriver/chromedriver")
driver.get(url)

path = driver.find_element_by_name("where")
path.clear()
sleep(2)
path.send_keys("Sydney")

for job in joi:
    elem = driver.find_element_by_name("keywords")
    elem.clear()
    sleep(2)
    elem.send_keys(job)
    elem.send_keys(Keys.RETURN)

#soup=BeautifulSoup(driver.page_source, 'lxml')

    sleep(4)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html')
    
    for item in soup.findAll('article', {'data-automation':'normalJob'}):
        extract_location_from_result(soup)
        extract_jobtitle_from_result(soup)
        extract_company_from_result(soup)
        extract_salary_from_result(soup)
        extract_links_from_result(soup)
    
    while True:
        try:
            elem = driver.find_element_by_class_name("bHpQ-bp")
            elem.send_keys(Keys.RETURN)
    
            sleep(5)
    
            html = driver.page_source
            soup = BeautifulSoup(html, 'html')
    
            for item in soup.findAll('article', {'data-automation':'normalJob'}):
        
                extract_location_from_result(soup)
                extract_jobtitle_from_result(soup)
                extract_company_from_result(soup)
                extract_links_from_result(soup)
                extract_salary_from_result(soup)
    
        except NoSuchElementException:
            break   
    
driver.quit()

len(links)

NoSuchWindowException: Message: no such window: window was already closed
  (Session info: chrome=72.0.3626.96)
  (Driver info: chromedriver=2.42.591059 (a3d9684d10d61aa0c45f6723b327283be1ebaad8),platform=Mac OS X 10.14.2 x86_64)


In [167]:
'''
Here I confirm that all lists are of the same length.
'''

print(len(title))
print(len(search))
print(len(company))
print(len(salary))
print(len(links))
print(len(location))

840
840
820
460
840
840


In [4]:
'''
Here I add the stem to the urls extracted from the search pages. This will allow another selenium scraper to go
through each page and pull out the job descriptions.
'''

stem = 'https://www.seek.com.au'
new_job_links = [stem + x for x in job_links]
len(new_job_links)

7317

In [5]:
new_job_links_dps = list(dict.fromkeys(new_job_links))
print(len(new_job_links), len(new_job_links_dps))

7317 7317


In [41]:
'''
Here I have created the second web scraper, which will crawl through the urls provided from section 1. I chose to do 
this as headless browser so I could keep working without the window continuously popping up.
Again, this took a really long time, so I don't recommend doing it this way.

'''


from selenium.webdriver.chrome.options import Options
chrome_options = Options()
chrome_options.add_argument("--headless")
driver = webdriver.Chrome(executable_path="./chromedriver/chromedriver", options = chrome_options)



for link in new_job_links_dps:
    url = link
    driver = webdriver.Chrome(executable_path="./chromedriver/chromedriver", options = chrome_options)
    driver.get(url)
    soup_level2=BeautifulSoup(driver.page_source, 'html')
    
    try:
        job_title.append(soup_level2.find_all('span',{'class':'lwHBT6d'})[0].text) if soup_level2.find('span',{'class':"lwHBT6d"}) else ''
        job_post_date.append(soup_level2.find_all('span',{'class':'lwHBT6d'})[1].text) if soup_level2.find('span',{'class':"lwHBT6d"}) else ''
        job_salary.append(soup_level2.find_all('span',{'class':"lwHBT6d"})[2].text) if soup_level2.find('span',{'class':"lwHBT6d"}) else ''
        job_desc.append(soup_level2.find('div',{'data-automation':"mobileTemplate"}).text) if soup_level2.find('div',{'data-automation':"mobileTemplate"}) else ''
        advertiser.append(soup_level2.find('span',{'class':"_3FrNV7v _2QG7TNq E6m4BZb"}).text) if soup_level2.find('span',{'class':"_3FrNV7v _2QG7TNq E6m4BZb"}) else ''
        print(link)
    except: 
        job_title.append('Missing')
        job_post_date.append('Missing')
        job_salary.append('Missing')
        job_desc.append('Missing')
        advertiser.append('Missing')
        continue
    driver.close()

print(job_title)



https://www.seek.com.au/job/38178403?type=promoted&searchrequesttoken=7b6249b9-6f2c-4a57-8278-38efccc36b2c
https://www.seek.com.au/job/38042307?type=promoted&searchrequesttoken=7b6249b9-6f2c-4a57-8278-38efccc36b2c
https://www.seek.com.au/job/38274601?type=standout&searchrequesttoken=7b6249b9-6f2c-4a57-8278-38efccc36b2c
https://www.seek.com.au/job/38267034?type=standout&searchrequesttoken=7b6249b9-6f2c-4a57-8278-38efccc36b2c
https://www.seek.com.au/job/38256490?type=standard&searchrequesttoken=7b6249b9-6f2c-4a57-8278-38efccc36b2c
https://www.seek.com.au/job/38248336?type=standard&searchrequesttoken=7b6249b9-6f2c-4a57-8278-38efccc36b2c
https://www.seek.com.au/job/38178979?type=standard&searchrequesttoken=7b6249b9-6f2c-4a57-8278-38efccc36b2c
https://www.seek.com.au/job/38214612?type=standout&searchrequesttoken=7b6249b9-6f2c-4a57-8278-38efccc36b2c
https://www.seek.com.au/job/38257609?type=standout&searchrequesttoken=7b6249b9-6f2c-4a57-8278-38efccc36b2c
https://www.seek.com.au/job/38194317?

https://www.seek.com.au/job/38262679?type=standard&searchrequesttoken=ddc02441-c339-4280-a6ac-614ae072dcb9
https://www.seek.com.au/job/38129847?type=standout&searchrequesttoken=ddc02441-c339-4280-a6ac-614ae072dcb9
https://www.seek.com.au/job/38165498?type=standard&searchrequesttoken=ddc02441-c339-4280-a6ac-614ae072dcb9
https://www.seek.com.au/job/38192445?type=standard&searchrequesttoken=ddc02441-c339-4280-a6ac-614ae072dcb9
https://www.seek.com.au/job/38268367?type=standard&searchrequesttoken=ddc02441-c339-4280-a6ac-614ae072dcb9
https://www.seek.com.au/job/38111478?type=standard&searchrequesttoken=ddc02441-c339-4280-a6ac-614ae072dcb9
https://www.seek.com.au/job/38269206?type=standard&searchrequesttoken=d8e06326-de9a-4f4c-90c5-014980e7d909
https://www.seek.com.au/job/38256071?type=standard&searchrequesttoken=d8e06326-de9a-4f4c-90c5-014980e7d909
https://www.seek.com.au/job/38203612?type=standout&searchrequesttoken=d8e06326-de9a-4f4c-90c5-014980e7d909
https://www.seek.com.au/job/38138486?

https://www.seek.com.au/job/38042307?type=standout&searchrequesttoken=3984f250-f0b0-4194-ab2c-2f0bcafc07a3
https://www.seek.com.au/job/38209840?type=standout&searchrequesttoken=3984f250-f0b0-4194-ab2c-2f0bcafc07a3
https://www.seek.com.au/job/38079957?type=standout&searchrequesttoken=3984f250-f0b0-4194-ab2c-2f0bcafc07a3
https://www.seek.com.au/job/38248187?type=standout&searchrequesttoken=3984f250-f0b0-4194-ab2c-2f0bcafc07a3
https://www.seek.com.au/job/38248125?type=standout&searchrequesttoken=3984f250-f0b0-4194-ab2c-2f0bcafc07a3
https://www.seek.com.au/job/38257913?type=standard&searchrequesttoken=3984f250-f0b0-4194-ab2c-2f0bcafc07a3
https://www.seek.com.au/job/38177851?type=standout&searchrequesttoken=3984f250-f0b0-4194-ab2c-2f0bcafc07a3
https://www.seek.com.au/job/38030526?type=standout&searchrequesttoken=3984f250-f0b0-4194-ab2c-2f0bcafc07a3
https://www.seek.com.au/job/38080178?type=standout&searchrequesttoken=3984f250-f0b0-4194-ab2c-2f0bcafc07a3
https://www.seek.com.au/job/38025135?

https://www.seek.com.au/job/38071618?type=standard&searchrequesttoken=c061b7ff-6ded-42ba-b5e5-d4d1cde487bf
https://www.seek.com.au/job/38156384?type=standout&searchrequesttoken=c061b7ff-6ded-42ba-b5e5-d4d1cde487bf
https://www.seek.com.au/job/38102934?type=standard&searchrequesttoken=c061b7ff-6ded-42ba-b5e5-d4d1cde487bf
https://www.seek.com.au/job/37753556?type=standard&searchrequesttoken=c061b7ff-6ded-42ba-b5e5-d4d1cde487bf
https://www.seek.com.au/job/38225817?type=standard&searchrequesttoken=c061b7ff-6ded-42ba-b5e5-d4d1cde487bf
https://www.seek.com.au/job/38121657?type=standard&searchrequesttoken=c061b7ff-6ded-42ba-b5e5-d4d1cde487bf
https://www.seek.com.au/job/38047623?type=standard&searchrequesttoken=c061b7ff-6ded-42ba-b5e5-d4d1cde487bf
https://www.seek.com.au/job/38136418?type=standard&searchrequesttoken=c061b7ff-6ded-42ba-b5e5-d4d1cde487bf
https://www.seek.com.au/job/38035497?type=standard&searchrequesttoken=c061b7ff-6ded-42ba-b5e5-d4d1cde487bf
https://www.seek.com.au/job/38139418?

https://www.seek.com.au/job/38269082?type=standout&searchrequesttoken=b9dded5f-640e-4f52-aa12-499ec587bb8a
https://www.seek.com.au/job/38107166?type=standard&searchrequesttoken=b9dded5f-640e-4f52-aa12-499ec587bb8a
https://www.seek.com.au/job/38082902?type=standard&searchrequesttoken=b9dded5f-640e-4f52-aa12-499ec587bb8a
https://www.seek.com.au/job/38226746?type=standard&searchrequesttoken=b9dded5f-640e-4f52-aa12-499ec587bb8a
https://www.seek.com.au/job/38155092?type=standard&searchrequesttoken=b9dded5f-640e-4f52-aa12-499ec587bb8a
https://www.seek.com.au/job/38273763?type=standout&searchrequesttoken=b9dded5f-640e-4f52-aa12-499ec587bb8a
https://www.seek.com.au/job/38130017?type=standout&searchrequesttoken=b9dded5f-640e-4f52-aa12-499ec587bb8a
https://www.seek.com.au/job/38274504?type=standard&searchrequesttoken=b9dded5f-640e-4f52-aa12-499ec587bb8a
https://www.seek.com.au/job/38273223?type=standard&searchrequesttoken=b9dded5f-640e-4f52-aa12-499ec587bb8a
https://www.seek.com.au/job/38137799?

...

https://www.seek.com.au/job/38188309?type=standard&searchrequesttoken=d625f490-94a6-4b84-baa8-77569a9d5998
https://www.seek.com.au/job/38116651?type=standout&searchrequesttoken=d625f490-94a6-4b84-baa8-77569a9d5998
https://www.seek.com.au/job/37662453?type=standout&searchrequesttoken=d625f490-94a6-4b84-baa8-77569a9d5998
https://www.seek.com.au/job/38121860?type=standout&searchrequesttoken=d625f490-94a6-4b84-baa8-77569a9d5998
https://www.seek.com.au/job/38224248?type=standard&searchrequesttoken=d625f490-94a6-4b84-baa8-77569a9d5998
https://www.seek.com.au/job/38031478?type=standard&searchrequesttoken=d625f490-94a6-4b84-baa8-77569a9d5998
https://www.seek.com.au/job/38179158?type=standard&searchrequesttoken=d625f490-94a6-4b84-baa8-77569a9d5998
https://www.seek.com.au/job/38157443?type=standard&searchrequesttoken=d625f490-94a6-4b84-baa8-77569a9d5998
https://www.seek.com.au/job/38181932?type=standout&searchrequesttoken=d625f490-94a6-4b84-baa8-77569a9d5998
https://www.seek.com.au/job/38085604?

https://www.seek.com.au/job/38158399?type=standard&searchrequesttoken=e6437c10-43af-4309-9e6b-7da138f87beb
https://www.seek.com.au/job/38176994?type=standard&searchrequesttoken=e6437c10-43af-4309-9e6b-7da138f87beb
https://www.seek.com.au/job/38083367?type=standout&searchrequesttoken=e6437c10-43af-4309-9e6b-7da138f87beb
https://www.seek.com.au/job/38127080?type=standout&searchrequesttoken=e6437c10-43af-4309-9e6b-7da138f87beb
https://www.seek.com.au/job/38129169?type=standout&searchrequesttoken=e6437c10-43af-4309-9e6b-7da138f87beb
https://www.seek.com.au/job/38150975?type=standard&searchrequesttoken=e6437c10-43af-4309-9e6b-7da138f87beb
https://www.seek.com.au/job/38151925?type=standard&searchrequesttoken=e6437c10-43af-4309-9e6b-7da138f87beb
https://www.seek.com.au/job/38256880?type=standout&searchrequesttoken=e6437c10-43af-4309-9e6b-7da138f87beb
https://www.seek.com.au/job/38159193?type=standard&searchrequesttoken=e6437c10-43af-4309-9e6b-7da138f87beb
https://www.seek.com.au/job/38261038?

https://www.seek.com.au/job/38036512?type=standard&searchrequesttoken=59b09af4-f659-4dd2-ab45-158f170c3641
https://www.seek.com.au/job/38065959?type=standard&searchrequesttoken=59b09af4-f659-4dd2-ab45-158f170c3641
https://www.seek.com.au/job/38228623?type=standard&searchrequesttoken=59b09af4-f659-4dd2-ab45-158f170c3641
https://www.seek.com.au/job/38155452?type=standout&searchrequesttoken=59b09af4-f659-4dd2-ab45-158f170c3641
https://www.seek.com.au/job/38170357?type=standard&searchrequesttoken=59b09af4-f659-4dd2-ab45-158f170c3641
https://www.seek.com.au/job/38238291?type=standard&searchrequesttoken=59b09af4-f659-4dd2-ab45-158f170c3641
https://www.seek.com.au/job/38097299?type=standard&searchrequesttoken=59b09af4-f659-4dd2-ab45-158f170c3641
https://www.seek.com.au/job/38176590?type=standout&searchrequesttoken=59b09af4-f659-4dd2-ab45-158f170c3641
https://www.seek.com.au/job/38182640?type=standard&searchrequesttoken=59b09af4-f659-4dd2-ab45-158f170c3641
https://www.seek.com.au/job/38261495?

https://www.seek.com.au/job/38195500?type=standout&searchrequesttoken=64d800a4-ab39-426a-a695-dc35aa952cb1
https://www.seek.com.au/job/38057016?type=standout&searchrequesttoken=64d800a4-ab39-426a-a695-dc35aa952cb1
https://www.seek.com.au/job/38040487?type=standard&searchrequesttoken=64d800a4-ab39-426a-a695-dc35aa952cb1
https://www.seek.com.au/job/38152970?type=standout&searchrequesttoken=64d800a4-ab39-426a-a695-dc35aa952cb1
https://www.seek.com.au/job/38057275?type=standout&searchrequesttoken=64d800a4-ab39-426a-a695-dc35aa952cb1
https://www.seek.com.au/job/38084864?type=standout&searchrequesttoken=64d800a4-ab39-426a-a695-dc35aa952cb1
https://www.seek.com.au/job/38131401?type=standard&searchrequesttoken=64d800a4-ab39-426a-a695-dc35aa952cb1
https://www.seek.com.au/job/38247373?type=standout&searchrequesttoken=64d800a4-ab39-426a-a695-dc35aa952cb1
https://www.seek.com.au/job/38121686?type=standard&searchrequesttoken=64d800a4-ab39-426a-a695-dc35aa952cb1
https://www.seek.com.au/job/38096460?

https://www.seek.com.au/job/38030637?type=standout&searchrequesttoken=3c9e69c4-c904-49bf-8ac5-576181230747
https://www.seek.com.au/job/38064800?type=standout&searchrequesttoken=c0b3be79-e1c3-46db-a7a4-1293ced9f40a
['Data Scientist', 'AngularJS Developer', 'Data Scientist', 'Data Science Delivery Lead', 'Data Science Leader ', 'Head of Data Science Engineering & Products', 'Machine Learning Engineer / Scientist - Optimisation experience', 'Recruitment Consultant | Data Science', 'Senior Data Science Executive', 'Data Analytics & Data Science Team Leader', 'Data Analytics & Data Science Team Leader', 'Junior Data Scientist', 'Data Scientist with Consultancy Experience', 'Head of Data Science Engineering & Products', 'Senior Data Science (Loyalty) :: $170K + Super + Bonus :: Sydney', 'Senior Data Scientist', 'Manager | Analytics & Data Science', 'Data Scientist l Machine Learning l $150,000 - $200,000 l CBD ', 'Data Scientist', 'Senior Data Scientist - Machine Learning', 'GHD Digital - An

In [114]:
'''
Here I create a dictionary from the lists where the web scraping data was stored to later convert to a DataFrame.
'''

data = {'job_title':job_title,
        'job_post_date':job_post_date,
        'job_salary':job_salary,
        'advertiser':advertiser,
        'job_desc':job_desc,
        'url':new_job_links_dps
       }


In [115]:
'''
And this is the DataFrame.
'''

seek = pd.DataFrame(data)


In [54]:
seek

,job_title,job_post_date,job_salary,job_desc
0,Data Scientist,23 Jan 2019,Full Time,ABOUT QHR/eQ8 year of providing consulting...
1,AngularJS Developer,8 Jan 2019,Full Time,Job Description Seeking an AngularJS developer...
2,Data Scientist,5 Feb 2019,Full Time,About the business and the role My client is ...
3,Data Science Delivery Lead,5 Feb 2019,"$150,000 - $180,000 package",About Intellify Intellify is a data science an...
4,Data Science Leader,4 Feb 2019,$180k - $210k base + super + bonus,New chance to be the number on Data Science ...
5,Head of Data Science Engineering & Products,1 Feb 2019,Full Time,Head of Data Science Engineering & Products - ...
6,Machine Learning Engineer / Scientist - Optimi...,23 Jan 2019,Circa upto 160k based on skills,This role is a newly created role to work with...
7,Recruitment Consultant | Data Science,29 Jan 2019,"Up to $90,000 + Super + Comms",Company Overview The global Digital transfo...
8,Senior Data Science Executive,4 Feb 2019,Full Time,Nielsen is a global performance management com...
9,Data Analytics & Data Science Team Leader,25 Jan 2019,"$110,172.00 - $119,921.00",The mission of the AFP is to provide dynamic a...


In [ ]:
'''
Here I exported the data to csv for subsequent processing.
'''

seek.to_csv('./seek.csv')